In [3]:
import Pkg
Pkg.add("Pkg")
Pkg.add("Dates")

guypath = "/Users/guyfarmer/ambulance_connect/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/"
michaelpath= "/home/mhilborn/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy"
joshpath = "/Users/Owner/Documents/Austin/simulateUpdate/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/"

currentpath = joshpath

cd(currentpath)
Pkg.activate(".")

using DataFrames, JLD, Distributions, CSV, Random, Plots,JSON, DelimitedFiles
import DataStructures: PriorityQueue, enqueue!, dequeue!

include(".//src//model.jl")

  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\simulateUpdate\AmbulanceDeployement\AmbulanceDeployment.jl-legacy\test\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\simulateUpdate\AmbulanceDeployement\AmbulanceDeployment.jl-legacy\test\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\simulateUpdate\AmbulanceDeployement\AmbulanceDeployment.jl-legacy\test\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\simulateUpdate\AmbulanceDeployement\AmbulanceDeployment.jl-legacy\test\Manifest.toml`
 Activating environment at `C:\Users\Owner\Documents\Austin\simulateUpdate\AmbulanceDeployement\AmbulanceDeployment.jl-legacy\Project.toml`
┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1278
┌ Info: Precompiling Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1278
┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ B

LoadError: LoadError: LoadError: [91mUndefVarError: Params not defined[39m
in expression starting at C:\Users\Owner\Documents\Austin\simulateUpdate\AmbulanceDeployement\AmbulanceDeployment.jl-legacy\src\deployment\robust.jl:29
in expression starting at C:\Users\Owner\Documents\Austin\simulateUpdate\AmbulanceDeployement\AmbulanceDeployment.jl-legacy\src\model.jl:86

"C:\\Users\\Owner\\Documents\\Austin\\simulateUpdate\\AmbulanceDeployement\\AmbulanceDeployment.jl-legacy"

In [7]:
"""Helper functions for the simuation engine?"""

#=
Author : Ng Yeesian
Modified : Guy Farmer / Zander Tedjo / Will Worthington / Michael Hilborn
generates the closest dispatch model
Summary :
Input :
Output :
=#
mutable struct ClosestDispatch <: DispatchModel
    drivetime::DataFrame
    candidates::Vector{Vector{Int}}
    available::Vector{Int}
    assignment::Vector{Int} # which location the ambulance is assigned to
    ambulances::Vector{Vector{Int}} # list of ambulances assigned to each location
    status::Vector{Symbol} # the current status of the ambulance
        # possible statuses: :available, :responding, :atscene, :conveying, :returning
    fromtime::Vector{Int} # the time it started the new status
    hospital::Vector{Int}
end

#=
Summary :
Input :
Output : ClosestDispatch struct
=#
function ClosestDispatch(p::DeploymentProblem, drivetime::DataFrame,distribution::Vector{Int})
    candidates = Any[]
    I = 1:p.nlocations
    for region in 1:p.nregions
        push!(candidates, I[vec(p.coverage[region,:])])
    end
    #println("creating the dispatch problem :$distribution")

    assignment = zeros(Int, p.nambulances)
    ambulances = [Int[] for i in 1:p.nlocations]
    k = 1
    for i in eachindex(distribution), j in 1:distribution[i]
        assignment[k] = i
        push!(ambulances[i], k)
        k += 1
    end
    status = fill(:available, p.nambulances)
    fromtime = zeros(Int, p.nambulances)
    hospital = zeros(Int, p.nambulances)
    
    ClosestDispatch(drivetime, candidates, distribution, assignment, ambulances, status, fromtime, hospital)

end

function ClosestDispatch(p::DeploymentProblem, problem::DispatchProblem)
    stn_names = [Symbol("stn$(i)_min") for i in 1:size(p.coverage,2)]
    ClosestDispatch(p, problem.emergency_calls[:, stn_names], problem.available)
end

function update_ambulances!(model::ClosestDispatch, i::Int, delta::Int)
    model.available[i] += delta
    @assert model.available[i] >= 0
end

function closest_available(dispatch::ClosestDispatch, id::Int, problem::DispatchProblem)
    location = 0
    min_time = Inf
    for i in dispatch.candidates[problem.emergency_calls[id, :neighborhood]]
        if problem.available[i] > 0 && dispatch.drivetime[id, i] < min_time
            #println("apparently this station is available : $i and here is the amount of ambulances avalable: $(problem.available[i])")
            location = i
            min_time = dispatch.drivetime[id, i]
        end
    end
    location
end

function respond_to!(dispatch::ClosestDispatch, i::Int, t::Int)
    @assert length(dispatch.ambulances[i]) > 0 "$i: $(dispatch.ambulances[i])"
    amb = popfirst!(dispatch.ambulances[i])
    # @assert redeploy.hospital[amb] == 0
    @assert amb != 0
    @assert dispatch.status[amb] == :available "$amb: $(dispatch.status[amb])"
    dispatch.status[amb] = :responding
    #dispatch.fromtime[amb] = t
    amb
end

function arriveatscene!(dispatch::ClosestDispatch, amb::Int, t::Int)
    @assert dispatch.status[amb] == :responding "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] == 0
    dispatch.status[amb] = :atscene
    dispatch.fromtime[amb] = t
end

function arriveathospital!(dispatch::DispatchModel, amb::Int, hosp::Int, t::Int)
    @assert dispatch.status[amb] == :atscene "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :atHospital
    dispatch.fromtime[amb] = t
    dispatch.hospital[amb] = hosp
end

function returning_to!(dispatch::DispatchModel, amb::Int, t::Int)
    @assert dispatch.status[amb] == :atHospital "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :returning
    dispatch.fromtime[amb] = t
    dispatch.assignment[amb]
end

returning_to! (generic function with 2 methods)

In [69]:
turnaround = Distributions.LogNormal(3.65, 0.3)
lambda = 0
local_path = ""
model_name = "Stochastic"
namb = 30
ncalls = 1000
#using AmbulanceDeployment

pwd()

cd(currentpath * "/test/")
pwd()

#using AmbulanceDeployment
using Dates

#include("..//src//dispatch/closestdispatch.jl")
include("..//src//problem.jl")
include("..//src//evaluate.jl")
#include("..//src//simulate.jl")

hospitals = CSV.File(string(local_path,"../test/austin-data/hospitals.csv")) |> DataFrame
stations = CSV.File(string(local_path,"../test/austin-data/stations.csv")) |> DataFrame
hourly_calls = CSV.File("../test/austin-data/Full_WeekdayCalls.csv") |> DataFrame
adjacent_nbhd = CSV.File("../test/austin-data/adjacent_nbhd.csv") |> DataFrame
coverage = CSV.read("../test/austin-data/coverage_real.csv", DataFrame, header=false)
coverage = convert(Array{Bool, 2}, coverage[:, :])
incidents = CSV.File("../test/austin-data/austin_incidents.csv") |> DataFrame
solverstats = JLD.load("../src/outputs/austin_team_stats_1_03.jld")
amb_deployment = solverstats["amb_deployment"]
model_dict = Dict{String, Symbol}("Stochastic"=>:Stochastic, "Robust01"=>:Robust01, "Robust005"=>:Robust005, "Robust001"=>:Robust001, "Robust0001"=>:Robust0001,
          "Robust00001"=>:Robust00001, "MEXCLP"=>:MEXCLP, "MALP"=>:MALP)


p = DeploymentProblem(
          hourly_calls,
          adjacent_nbhd,
          coverage,
          namb = namb,
          train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
          )
test_calls = CSV.File("../test/austin-data/austin_test_calls.csv")|> DataFrame
test_calls = test_calls[1:ncalls,:]
println("running $namb ambuances & $ncalls calls")

# distribution of ambulances across all of the stations
x = amb_deployment[model_dict[model_name]][namb]

#establishes the dispatch problem
problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

#establishes the dispatch model
dispatch = ClosestDispatch(p, problem)

Random.seed!(1234)



running 30 ambuances & 1000 calls


MersenneTwister(UInt32[0x000004d2], Random.DSFMT.DSFMT_state(Int32[-1393240018, 1073611148, 45497681, 1072875908, 436273599, 1073674613, -2043716458, 1073445557, -254908435, 1072827086  …  -599655111, 1073144102, 367655457, 1072985259, -1278750689, 1018350124, -597141475, 249849711, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000000

In [70]:
using AmbulanceDeployment, Dates

In [71]:
Pkg.add("Query")
using Query

struct EMSEngine{T}
    eventlog::DataFrame
    eventqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    shortqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    guiArray::Array{Any, 1}
end

struct gui_event
    #call made / call responded / ambulance arrived / shortfall
    event_type::String
    #neighborhood where call is made from
    neighborhood_id::Int
    # station where call is made from
    deployment_id::Int
    # id of the event generated
    event_id::Int
    ambulance_id::Int
    arrival_time::Int
    remaining_amb::Int
    # current timestamp of event
    timestamp::DateTime
end

## generates the priority queue and instantiates the EMSEngine struct
function generateCalls(problem::DispatchProblem)
    ncalls = nrow(problem.emergency_calls)
    eventlog = DataFrame(
        id = 1:ncalls,
        dispatch_from = zeros(Int, ncalls),
        waittime = fill(0.0, ncalls),
        responsetime = fill(Inf, ncalls),
        scenetime = fill(Inf, ncalls),
        conveytime = fill(Inf, ncalls),
        returntime = fill(Inf, ncalls),
        return_to = zeros(Int, ncalls),
        return_type = fill(:station, ncalls),
        hospital = zeros(Int, ncalls),
        ambulance = zeros(Int, ncalls)
    )
    eventqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    shortqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    guiArray = Any[]
    for i in 1:nrow(problem.emergency_calls)
        t = problem.emergency_calls[i, :arrival_seconds]
        enqueue!(eventqueue, (:call, i, t, problem.emergency_calls[i, :neighborhood]), t)
    end
    EMSEngine{Tuple{Symbol,Int,Int,Int}}(eventlog, eventqueue, shortqueue, guiArray) ## returns the ems engine struct
end

function simulate_events!(problem::DispatchProblem, dispatch::ClosestDispatch)
    engine = generateCalls(problem)
    while(~isempty(engine.eventqueue))
        (event, id, t, value) = dequeue!(engine.eventqueue)

        if event == :call
            # if the event is reachable then change event to station2call
            # if not it is a shortfall
            println("calling event id: $id time: $t value: $value")
            call_event!(engine, problem, dispatch, id, t, value)
           
        elseif event == :station2call
            #once the ambulance arrives at the station change event to call2hospital
            println("Ambulance has arrived on the scene for event $id at time $t")
            station2call_event!(engine, problem, dispatch, id, t, value)
        elseif event == :call2hospital
            #once the ambulance arrives at the hospital it will hand over the patient and then 
            #change the deploy back to its station
            println("Ambulance for event $id has arrived at the hospital at time $t")
            call2hosp_event!(engine, problem, dispatch, id, t, value)

        elseif event == :hospital2station
            #once the ambulance drops off the patient it will reroute to the station
            println("Ambulance for event $id is returning to the station at $t")
            return_event!(engine, problem, dispatch, id, t, value)
            
        elseif event == :done
            #increase the number of ambulances at that station now that it is done
            done_event!(engine, problem, dispatch, id, t, value)
            
        end
    end
    println("This is the Short Queue")
    while(~isempty(engine.shortqueue))
       (event, id, t, value) = dequeue!(engine.shortqueue)
        print("$event, $id, $t, $value")
        print("This is not empty")
    end
    engine.eventlog , engine.guiArray
end


function call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        nbhd::Int; # the neighborhood the call is from
    )

    #check if there is an ambulance within the coverage matrix
    if sum(problem.deployment[problem.coverage[nbhd,:]]) == 0
        @assert false "$id: no ambulance reachable for call at $nbhd"
    #check if one of the ambulances within the coverage is available
    elseif sum(problem.available[problem.coverage[nbhd,:]]) > 0
        i = closest_available(dispatch, id, problem)
        #println("closest available station : $i")
        @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
        update_ambulances!(dispatch, i, -1)
        ems.eventlog[id, :dispatch_from] = i
        @assert problem.available[i] >= 0
        

        travel_time = ceil(Int, 60*2*problem.emergency_calls[id, Symbol("stn$(i)_min")])
        @assert travel_time >= 0
        if(t != problem.emergency_calls[id, :arrival_seconds])
            print("This is a shortfall")
        end
        ems.eventlog[id, :responsetime] = (travel_time + t - problem.emergency_calls[id,:arrival_seconds]) / 60 # minutes

        amb = respond_to!(dispatch, i, t)
        ems.eventlog[id, :ambulance] = amb

        event = gui_event("call responded", problem.emergency_calls[id, :neighborhood], i, id, amb,-1, problem.available[i] ,Dates.now())
        num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
        for i in 1:size(problem.available, 1)
            num_ambulances_array[i] = problem.available[i]
        end
        push!(ems.guiArray,event)
        push!(ems.guiArray,num_ambulances_array)
        enqueue!(ems.eventqueue, (:station2call, id, t + travel_time, amb), t + travel_time)
    #else queue it
    else
        # println(id, ": call from ", nbhd, " queued behind ", problem.wait_queue[nbhd])
        # event = gui_event("call made", nbhd, -1, id, -1,-1,-1,Dates.now())
        # push!(ems.guiArray,event)
        # problem.shortfalls = problem.shortfalls + 1
        # #push!(problem.shortfalls) # count shortfalls
        # push!(problem.wait_queue[nbhd], id) # queue the emergency call
        event = gui_event("shortfall", problem.emergency_calls[id, :neighborhood], -1, id, -1,-1,-1 ,Dates.now())
        push!(ems.guiArray, event)
        enqueue!(ems.shortqueue, (:call, id, t, problem.emergency_calls[id, :neighborhood]), t)
        println(id, ": call from ", nbhd, " is a shortfall")
    end
end

function station2call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    arriveatscene!(dispatch, amb, t)
    # time the ambulance spends at the scene
    scene_time = ceil(Int,60*0.4*rand(problem.turnaround)) # 60sec*0.4*mean(40minutes) ~ 15minutes
    ems.eventlog[id, :scenetime] = scene_time / 60 # minutes
    @assert scene_time > 0
    
    event = gui_event("ambulance arrived", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    enqueue!(ems.eventqueue, (:call2hospital, id, t + scene_time, amb), t + scene_time)
end

function call2hosp_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    h = let mintime = Inf, minindex = 0
        for h in 1:nrow(problem.hospitals)
            traveltime = problem.emergency_calls[id, Symbol("hosp$(h)_min")]
            if !isna(traveltime) && !(traveltime=="NA") && traveltime < mintime
                @assert traveltime >= 0
                minindex = h; mintime = traveltime
            end
        end
        minindex
    end
    @assert h != 0
    dispatch.hospital[amb] = ems.eventlog[id, :hospital] = h
    conveytime = 60*15 + ceil(Int, 60*problem.emergency_calls[id, Symbol("hosp$(h)_min")]) # ~20minutes
    ems.eventlog[id, :conveytime] = conveytime / 60 # minutes
    @assert conveytime >= 0 conveytime
    arriveathospital!(dispatch, amb, h, t)
    
    #         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    event = gui_event("ambulance hospital", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:hospital2station, id, t+conveytime, amb), t+conveytime)
end

function return_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    stn = returning_to!(dispatch, amb, t)
    h = dispatch.hospital[amb]
    returntime = ceil(Int,60*2*problem.hospitals[h, Symbol("stn$(stn)_min")]) # ~ 10minutes
    dispatch.hospital[amb] = 0
    ems.eventlog[id, :returntime] = returntime / 60 # minutes
    @assert returntime >= 0 returntime
    t_end = t + returntime
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returning", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:done, id, t_end, amb), t_end)
    # num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
    # for i in 1:size(problem.available, 1)
    #     num_ambulances_array[i] = problem.available[i]
    # end
    # push!(ems.guiArray,num_ambulances_array)
end

function done_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    i = dispatch.assignment[amb]
    println("Ambulance from event $id has returned to station $i")
    @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
    update_ambulances!(dispatch, i, 1)
    push!(dispatch.ambulances[i], amb)
    dispatch.status[amb] = :available
    ems.eventlog[id, :return_to] = id
    @assert problem.available[i] > 0
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returned", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    while(~isempty(ems.shortqueue))
        # record time difference
        (event, short_id, short_t, value) = dequeue!(ems.shortqueue)
        enqueue!(ems.eventqueue, (event, short_id, t , value), short_t)
    end
end


  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\simulateUpdate\AmbulanceDeployement\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\simulateUpdate\AmbulanceDeployement\AmbulanceDeployment.jl-legacy\Manifest.toml`


done_event! (generic function with 1 method)

In [72]:
@time df, guiArray = simulate_events!(problem, dispatch)

calling event id: 1 time: 38 value: 83
calling event id: 2 time: 575 value: 38
Ambulance has arrived on the scene for event 1 at time 638
calling event id: 3 time: 673 value: 129
calling event id: 4 time: 691 value: 107
calling event id: 5 time: 1166 value: 113
calling event id: 6 time: 1187 value: 51
calling event id: 7 time: 1450 value: 76
calling event id: 8 time: 1578 value: 67
calling event id: 9 time: 1640 value: 52
Ambulance has arrived on the scene for event 6 at time 1787
Ambulance for event 1 has arrived at the hospital at time 1836
calling event id: 10 time: 1838 value: 67
calling event id: 11 time: 1899 value: 67
calling event id: 12 time: 1901 value: 82
Ambulance has arrived on the scene for event 4 at time 1912
Ambulance has arrived on the scene for event 2 at time 1924
calling event id: 13 time: 2091 value: 98
calling event id: 14 time: 2117 value: 113
Ambulance has arrived on the scene for event 8 at time 2178
Ambulance has arrived on the scene for event 3 at time 2209


This is a shortfallAmbulance for event 29 is returning to the station at 10649
Ambulance has arrived on the scene for event 35 at time 10685
Ambulance for event 21 is returning to the station at 10713
calling event id: 39 time: 10727 value: 52
39: call from 52 is a shortfall
calling event id: 40 time: 10770 value: 98
40: call from 98 is a shortfall
Ambulance has arrived on the scene for event 34 at time 10880
Ambulance has arrived on the scene for event 36 at time 11013
Ambulance for event 31 is returning to the station at 11013
Ambulance for event 27 is returning to the station at 11076
Ambulance for event 26 is returning to the station at 11232
Ambulance for event 32 has arrived at the hospital at time 11372
calling event id: 41 time: 11386 value: 128
Ambulance for event 33 has arrived at the hospital at time 11407
Ambulance from event 27 has returned to station 17
calling event id: 39 time: 11483 value: 52
39: call from 52 is a shortfall
calling event id: 40 time: 11483 value: 98
Th

49: call from 84 is a shortfall
Ambulance has arrived on the scene for event 55 at time 17063
Ambulance for event 57 has arrived at the hospital at time 17095
Ambulance from event 44 has returned to station 9
calling event id: 49 time: 17096 value: 84
49: call from 84 is a shortfall
calling event id: 59 time: 17165 value: 98
Ambulance for event 53 has arrived at the hospital at time 17172
calling event id: 60 time: 17321 value: 98
Ambulance has arrived on the scene for event 50 at time 17346
Ambulance has arrived on the scene for event 56 at time 17656
Ambulance for event 52 has arrived at the hospital at time 17670
Ambulance has arrived on the scene for event 58 at time 17735
Ambulance has arrived on the scene for event 59 at time 17765
Ambulance for event 48 is returning to the station at 17805
Ambulance for event 55 has arrived at the hospital at time 17815
Ambulance for event 51 is returning to the station at 17862
Ambulance for event 54 is returning to the station at 17942
Ambulan

This is a shortfallcalling event id: 79 time: 25449 value: 98
79: call from 98 is a shortfall
Ambulance from event 61 has returned to station 40
calling event id: 79 time: 25548 value: 98
This is a shortfallAmbulance has arrived on the scene for event 77 at time 25688
Ambulance has arrived on the scene for event 76 at time 25760
Ambulance has arrived on the scene for event 74 at time 25928
Ambulance for event 75 has arrived at the hospital at time 25961
calling event id: 80 time: 25961 value: 52
Ambulance for event 72 is returning to the station at 26113
Ambulance has arrived on the scene for event 79 at time 26148
Ambulance from event 71 has returned to station 1
Ambulance for event 77 has arrived at the hospital at time 26426
Ambulance for event 74 has arrived at the hospital at time 26502
calling event id: 81 time: 26594 value: 114
81: call from 114 is a shortfall
calling event id: 82 time: 26642 value: 98
calling event id: 83 time: 26657 value: 115
Ambulance for event 79 has arrive

Ambulance has arrived on the scene for event 102 at time 37606
Ambulance for event 100 has arrived at the hospital at time 37649
Ambulance from event 94 has returned to station 8
Ambulance for event 96 is returning to the station at 37855
calling event id: 105 time: 37866 value: 67
Ambulance for event 95 is returning to the station at 37897
Ambulance for event 101 is returning to the station at 37932
Ambulance for event 98 is returning to the station at 37945
Ambulance for event 103 has arrived at the hospital at time 37997
Ambulance for event 97 is returning to the station at 38146
Ambulance from event 95 has returned to station 40
calling event id: 106 time: 38218 value: 128
Ambulance from event 96 has returned to station 17
calling event id: 107 time: 38365 value: 52
Ambulance has arrived on the scene for event 105 at time 38466
Ambulance for event 99 is returning to the station at 38555
calling event id: 108 time: 38666 value: 128
Ambulance has arrived on the scene for event 104 at

...

578: call from 143 is a shortfall
calling event id: 565 time: 163547 value: 82
565: call from 82 is a shortfall
calling event id: 575 time: 163547 value: 113
575: call from 113 is a shortfall
calling event id: 542 time: 163547 value: 83
542: call from 83 is a shortfall
calling event id: 563 time: 163547 value: 113
563: call from 113 is a shortfall
calling event id: 579 time: 163547 value: 98
579: call from 98 is a shortfall
Ambulance from event 551 has returned to station 17
calling event id: 541 time: 163548 value: 129
541: call from 129 is a shortfall
calling event id: 579 time: 163548 value: 98
This is a shortfallcalling event id: 570 time: 163548 value: 98
570: call from 98 is a shortfall
calling event id: 520 time: 163548 value: 99
520: call from 99 is a shortfall
calling event id: 539 time: 163548 value: 129
539: call from 129 is a shortfall
calling event id: 573 time: 163548 value: 83
573: call from 83 is a shortfall
calling event id: 560 time: 163548 value: 83
560: call from 83

507: call from 98 is a shortfall
calling event id: 577 time: 164231 value: 113
577: call from 113 is a shortfall
calling event id: 578 time: 164231 value: 143
578: call from 143 is a shortfall
calling event id: 563 time: 164231 value: 113
563: call from 113 is a shortfall
calling event id: 520 time: 164231 value: 99
520: call from 99 is a shortfall
calling event id: 512 time: 164231 value: 126
512: call from 126 is a shortfall
calling event id: 581 time: 164231 value: 128
581: call from 128 is a shortfall
calling event id: 575 time: 164231 value: 113
575: call from 113 is a shortfall
calling event id: 542 time: 164231 value: 83
542: call from 83 is a shortfall
calling event id: 583 time: 164231 value: 141
583: call from 141 is a shortfall
Ambulance has arrived on the scene for event 547 at time 164292
Ambulance for event 561 has arrived at the hospital at time 164366
Ambulance for event 574 has arrived at the hospital at time 164439
Ambulance has arrived on the scene for event 580 at t

Excessive output truncated after 524297 bytes.

: call from 129 is a shortfall

(1000×11 DataFrame. Omitted printing of 6 columns
│ Row  │ id    │ dispatch_from │ waittime │ responsetime │ scenetime │
│      │ Int64 │ Int64         │ Float64  │ Float64      │ Float64   │
├──────┼───────┼───────────────┼──────────┼──────────────┼───────────┤
│ 1    │ 1     │ 1             │ 0.0      │ 10.0         │ 19.9667   │
│ 2    │ 2     │ 43            │ 0.0      │ 22.4833      │ 11.75     │
│ 3    │ 3     │ 25            │ 0.0      │ 25.6         │ 29.8833   │
│ 4    │ 4     │ 34            │ 0.0      │ 20.35        │ 13.2833   │
│ 5    │ 5     │ 17            │ 0.0      │ 19.5333      │ 18.0667   │
│ 6    │ 6     │ 22            │ 0.0      │ 10.0         │ 11.75     │
│ 7    │ 7     │ 39            │ 0.0      │ 49.0833      │ 26.8333   │
│ 8    │ 8     │ 8             │ 0.0      │ 10.0         │ 19.95     │
│ 9    │ 9     │ 22            │ 0.0      │ 30.55        │ 16.0      │
│ 10   │ 10    │ 24            │ 0.0      │ 19.2333      │ 17.9      │
⋮
│ 990  │ 990   │ 43      

In [54]:
guiArray

6452-element Array{Any,1}:
 gui_event("call responded", 83, 1, 1, 1, -1, 0, DateTime("2021-04-03T20:25:24.288"))
 Integer[0, 4, 1, 1, 0, 0, 1, 3, 0, 0  …  0, 1, 0, 0, 2, 0, 1, 0, 2, 1]
 gui_event("call responded", 38, 43, 2, 48, -1, 1, DateTime("2021-04-03T20:25:24.304"))
 Integer[0, 4, 1, 1, 0, 0, 1, 3, 0, 0  …  0, 1, 0, 0, 2, 0, 1, 0, 1, 1]
 gui_event("ambulance arrived", 83, -1, 1, 1, 638, -1, DateTime("2021-04-03T20:25:24.32"))
 gui_event("call responded", 129, 36, 3, 44, -1, 0, DateTime("2021-04-03T20:25:24.335"))
 Integer[0, 4, 1, 1, 0, 0, 1, 3, 0, 0  …  0, 0, 0, 0, 2, 0, 1, 0, 1, 1]
 gui_event("call responded", 107, 34, 4, 42, -1, 1, DateTime("2021-04-03T20:25:24.351"))
 Integer[0, 4, 1, 1, 0, 0, 1, 3, 0, 0  …  0, 0, 0, 0, 2, 0, 1, 0, 1, 1]
 gui_event("call responded", 113, 14, 5, 14, -1, 0, DateTime("2021-04-03T20:25:24.367"))
 Integer[0, 4, 1, 1, 0, 0, 1, 3, 0, 0  …  0, 0, 0, 0, 2, 0, 1, 0, 1, 1]
 gui_event("call responded", 51, 22, 6, 26, -1, 0, DateTime("2021-04-03T20:25:24.

In [73]:
mean(df["responsetime"])

62.19070000000001

In [74]:
using DelimitedFiles
filename = string(model_name,string(namb),"calls",string(ncalls),".csv")
writedlm(filename,  df["responsetime"])
print(filename)

Stochastic30calls1000.csv

"40"